
# Perform a minimal SCA analysis and plot SCA-related heatmaps

This example shows how to generate a SCA co-evolutionary matrix,
reduce it, and sort it according to XCoRs.


In [ ]:
# Author: Margaux Jullien <margaux.jullien@univ-grenoble-alpes.fr>
#         Nelle Varoquaux <nelle.varoquaux@univ-grenoble-alpes.fr>
#         Ivan Junier <ivan.junier@univ-grenoble-alpes.fr>
# License: TBD

Necessary imports



In [ ]:
import numpy as np
import matplotlib.pyplot as plt


import cocoatree.datasets as c_data
import cocoatree.msa as c_msa
import cocoatree.deconvolution as c_deconv

from cocoatree.statistics.pairwise import compute_sca_matrix

## Loading the dataset
For more details on the S1A serine proteases dataset, go to
`sphx_glr_auto_examples_d_datasets_plot_s1A_serine_proteases.py`.



In [ ]:
serine_dataset = c_data.load_S1A_serine_proteases()
loaded_seqs = serine_dataset["alignment"]
loaded_seqs_id = serine_dataset["sequence_ids"]
n_loaded_pos, n_loaded_seqs = len(loaded_seqs[0]), len(loaded_seqs)

## Filtering the MSA



In [ ]:
seq_kept, seq_id_kept, pos_kept = c_msa.filter_sequences(loaded_seqs,
                                                         loaded_seqs_id)

## Computing the SCA matrix



In [ ]:
SCA_matrix = compute_sca_matrix(seq_kept)

## Plotting the SCA matrix



In [ ]:
fig, ax = plt.subplots()
im = ax.imshow(SCA_matrix, vmin=0, vmax=2, cmap='inferno')

ax.set_xlabel('residues', fontsize=10)
ax.set_ylabel(None)
ax.set_title('SCA matrix')
fig.colorbar(im, shrink=0.7)

## Extracting XCoRs



In [ ]:
n_xcors = 3
xcors = c_deconv.extract_xcors(SCA_matrix, n_xcors=n_xcors)

print('XCoR positions on (filtered) sequences:')
for ixcor, xcor in enumerate(xcors):
    print('XCoR %d: %s' % (ixcor+1, xcor))

A plotting function returning a reduced coevo matrix, keeping
only XCoRs and sorting the residues according to them
-------------------------------------------------------------



In [ ]:
def plot_coevo_according2xcors(coevo_matrix, xcors=[], vmin=0, vmax=1e6):
    fig, ax = plt.subplots(tight_layout=True)

    xcor_sizes = [len(x) for x in xcors]
    cumul_sizes = sum(xcor_sizes)
    sorted_pos = [p for xcor in xcors for p in xcor]

    im = ax.imshow(coevo_matrix[np.ix_(sorted_pos, sorted_pos)],
                   vmin=vmin, vmax=vmax,
                   interpolation='none', aspect='equal',
                   extent=[0, cumul_sizes, 0, cumul_sizes], cmap='inferno')
    cb = fig.colorbar(im)
    cb.set_label("coevolution level")

    line_index = 0
    for i in range(n_xcors):
        ax.plot([line_index + xcor_sizes[i], line_index + xcor_sizes[i]],
                [0, cumul_sizes], 'w', linewidth=2)
        ax.plot([0, cumul_sizes], [cumul_sizes - line_index,
                                   cumul_sizes - line_index], 'w', linewidth=2)
        line_index += xcor_sizes[i]

    return fig, ax

## Plotting the SCA matrix according to the XCoRs



In [ ]:
fig, ax = plot_coevo_according2xcors(SCA_matrix, xcors, vmin=0, vmax=2)
ax.set_title('SCA matrix, sorted according to XCoRs')
ax.set_xlabel('XCoR\'s positions', fontsize=10)

Removing a global mode (ngm = no global mode),
i.e., setting largest eigeinvalue to zero



In [ ]:
SCA_matrix_ngm = c_deconv.remove_global_correlations(SCA_matrix)

## Plotting the SCA matrix without global mode according to the XCoRs



In [ ]:
fig, ax = plot_coevo_according2xcors(SCA_matrix_ngm, xcors, vmin=0, vmax=2)
ax.set_title('SCA matrix without global mode\nsorted according to XCoRs')
ax.set_xlabel('XCoR\'s positions', fontsize=10)

## Adapting the coevolution scale



In [ ]:
fig, ax = plot_coevo_according2xcors(SCA_matrix_ngm, xcors, vmin=0, vmax=1)
ax.set_title('SCA matrix without global mode\nsorted according to XCoRs')
ax.set_xlabel('XCoR\'s positions', fontsize=10)